This file will be used to contain data processing components

In [45]:
%pip install boto3 pyspark delta-spark python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [46]:
import os, boto3
from pyspark.sql import SparkSession

In [47]:
# You can use the following to set the environment variables in the notebook if you don't set manually access key, secret key and endpoint in minio
os.environ['OBJ_STORAGE_ACCESS_KEY'] = 'tMveEaHOQDdrH8Vt8GgJ'
os.environ['OBJ_STORAGE_SECRET_KEY'] = 'CY2Ze1R9HMoC0igRj8bOTL9qVk9OcMt6Z6jZ3yZy'
os.environ['OBJ_STORAGE_ENDPOINT'] = 'http://localhost:9000'

In [48]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')

In [49]:
path_1 = "s3a://data/data-raw/data.json"
path_2 = "s3a://data/data-raw/data2.json"
path_3 = "s3a://data/data-raw/data3.json"

In [50]:
# 6/ Write code: Use Spark to connect to Minio to read the data.
# You need to more configuration if you want to use minio as object storage 
# (hint: maybe you can using .config() method to set the configuration if you want using spark to read/write data from/to minio)
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config('spark.hadoop.fs.s3a.impl', "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config('spark.hadoop.fs.s3a.path.style.access', "true") \
    .config("spark.hadoop.fs.impl.disable.cache.prefetching", "true") \
    .config("spark.hadoop.fs.s3a.access.key", obj_storage_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", obj_storage_secret_key) \
    .config("spark.hadoop.fs.s3a.endpoint", obj_storage_endpoint) \
    .getOrCreate()

data = spark.read.format("json").load([path_1,path_2,path_3], multiLine=True)
data.printSchema()
data.show()

root
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- filling: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)

+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-----------------

In [51]:
# 7/ Write code: Use Spark to merge the files data.json, data2.json, data3.json into a single file named result.json => Then, upload it to the "data-result" folder on Minio.
# Cach 1 
#file_path = "s3a://data/data-raw/*"
bucket_name = "data"
folder_path = "data-result"
result_file = "result.json"
file_path = [path_1, path_2, path_3]
data = spark.read.format("json").load(file_path, multiLine=True)
data.printSchema()
data.show()
data.coalesce(1).write.json(f"s3a://{bucket_name}/{folder_path}/{result_file}", mode="overwrite")



root
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- filling: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)

+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-----------------

In [52]:
# 7/ Write code: Use Spark to merge the files data.json, data2.json, data3.json into a single file named result.json => Then, upload it to the "data-result" folder on Minio.
# Cach 2
"""
paths = [path_1, path_2, path_3]
merged_df = None
df = None
for path in paths:
    df = spark.read.format("json").load(path, multiLine=True)
    if merged_df is None:
        merged_df = df
    else:
        merged_df = merged_df.unionByName(df, allowMissingColumns=True)
merged_df.show()
output_path = "result.json"
merged_df.coalesce(1).write.json("s3a://data/data-result/result.json", mode="overwrite")
"""



'\npaths = [path_1, path_2, path_3]\nmerged_df = None\ndf = None\nfor path in paths:\n    df = spark.read.format("json").load(path, multiLine=True)\n    if merged_df is None:\n        merged_df = df\n    else:\n        merged_df = merged_df.unionByName(df, allowMissingColumns=True)\nmerged_df.show()\noutput_path = "result.json"\nmerged_df.coalesce(1).write.json("s3a://data/data-result/result.json", mode="overwrite")\n'

In [53]:
# 8/ Write code: Then, use Spark to read the result.json file => Display the data => You might have some duplicate data, please remove them => Display the data again after processing (Please retain the key steps you used to derive the final result).

# Step 1: Read the result.json file
result_df = spark.read.json(f"s3a://{bucket_name}/{folder_path}/{result_file}/*.json")
# Step 2: Display the data before removing duplicates
print(f"Len before processing: {result_df.count()}")
result_df.show()
# Step 3: Remove duplicate data
print("Remove duplicate data")
df_remove_duplicate = result_df.dropDuplicates()
# Step 4: Display the data after removing duplicates
print(f"Len after processing: {df_remove_duplicate.count()}")
df_remove_duplicate.show()


Len before processing: 19
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|{[{1001, Regular}...|                NULL|0004|              Jelly|0.65|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0005|     Custard-Filled|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|[{6001, None}, {6...|0006|     Cinnamon Twist|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0007|    Vanilla Frosted|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0008| Strawberry Frosted|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0009|     Chocolate Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0010|     Blueberry Ca

In [54]:
print(df_remove_duplicate.schema)

StructType([StructField('batters', StructType([StructField('batter', ArrayType(StructType([StructField('id', StringType(), True), StructField('type', StringType(), True)]), True), True)]), True), StructField('filling', ArrayType(StructType([StructField('id', StringType(), True), StructField('name', StringType(), True)]), True), True), StructField('id', StringType(), True), StructField('name', StringType(), True), StructField('ppu', DoubleType(), True), StructField('topping', ArrayType(StructType([StructField('id', StringType(), True), StructField('type', StringType(), True)]), True), True), StructField('type', StringType(), True)])


In [55]:
# 9/ Check your results. Before eliminating duplicates it will be 19 and after duplicates will it be equal to 18? The final result is the csv file uploaded to minio or not and can the csv file be read or not.
path_csv = f"s3a://{bucket_name}/{folder_path}/{result_file}/output.csv"
from pyspark.sql.functions import col, to_json
df_flat = df_remove_duplicate.withColumn("batters", to_json(col("batters"))) \
    .withColumn("filling", to_json(col("filling"))) \
    .withColumn("topping", to_json(col("topping")))
df_flat.write.csv(path_csv, header=True, mode="overwrite")
df_read_back = spark.read.csv(path_csv)
df_read_back.show()


+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|                 _c0|                 _c1| _c2|                _c3| _c4|                 _c5|  _c6|
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
|{"batter":[{"id":...|                NULL|0018|            Regular|0.55|[{"id":"5001","ty...|donut|
|{"batter":[{"id":...|                NULL|0021|      Vanilla Cream|0.75|[{"id":"5002","ty...|donut|
|{"batter":[{"id":...|[{"id":"6001","na...|0016|       Filled Donut|0.95|[{"id":"5001","ty...|donut|
|{"batter":[{"id":...|[{"id":"6001","na...|0019|          Blueberry|0.75|[{"id":"5002","ty...|donut|
|{"batter":[{"id":...|                NULL|0004|              Jelly|0.65|[{"id":"5001","ty...|donut|
|{"batter":[{"id":...|                NULL|0011|  Devil's Food Cake|0.75|[{"id":"5001","ty.